In [1]:
from torch.utils.data import DataLoader
from torchgeo.datasets import VHR10
 
dataset = VHR10(root="...", download=True, checksum=True)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True, num_workers=4)
 
for batch in dataloader:
    image = batch["image"]
    label = batch["label"]
 
    # train a model, or make predictions using a pre-trained model

URLError: <urlopen error [Errno 101] Network is unreachable>

In [2]:
import torch

# assume 'x' is your input tensor with shape (32, 1, 2560)
x = torch.randn(32, 1, 2560)

# perform PCA on the input tensor
components, explained_variance = torch.pca_lowrank(x, q=1280)

# reconstruct the reduced tensor
x_reduced = torch.matmul(x, components)

print(x_reduced.shape)  # should be (32, 1, 1280)

ValueError: q(=1280) must be non-negative integer and not greater than min(m, n)=1

In [5]:
import torch

# assume 'x' is your input tensor with shape (32, 1, 2560)
x = torch.randn(32, 1, 2560)

# perform SVD (PCA) on the input tensor
u, s, v = torch.svd(x)

# select the top 1280 singular values and corresponding singular vectors
u_reduced = u[:, :, :1280]
s_reduced = s[:, :1280]
v_reduced = v[:, :1280, :]

# reconstruct the reduced tensor
x_reduced = torch.matmul(u_reduced, torch.diag_embed(s_reduced))
print(x_reduced.shape)  # should be (32, 1, 1280)
x_reduced = torch.matmul(x_reduced, v_reduced)

print(x_reduced.shape)  # should be (32, 1, 1280)

torch.Size([32, 1, 1])


RuntimeError: Expected size for first two dimensions of batch2 tensor to be: [32, 1] but got: [32, 1280].

In [6]:
import torch

def pca_reduction(data, n_components):
    # 标准化数据
    data_mean = torch.mean(data, dim=0, keepdim=True)
    data_std = torch.std(data, dim=0, keepdim=True)
    data_normalized = (data - data_mean) / data_std

    # 计算协方差矩阵
    cov_matrix = torch.cov(data_normalized.view(-1, data.size(-1)).T)

    # 计算特征值和特征向量
    eigenvalues, eigenvectors = torch.linalg.eigh(cov_matrix)

    # 选择前 n_components 个特征向量
    top_eigenvectors = eigenvectors[:, -n_components:]

    # 投影数据
    reduced_data = torch.matmul(data_normalized.view(-1, data.size(-1)), top_eigenvectors)

    # 调整形状为 (32, 1, 100)
    reduced_data = reduced_data.view(data.size(0), data.size(1), n_components)

    return reduced_data

# 假设你有一个形状为 (32, 1, 240) 的张量
input_tensor = torch.randn(32, 1, 240)

# 进行 PCA 降维到 100 维
output_tensor = pca_reduction(input_tensor, n_components=100)

print(output_tensor.shape)  # 输出应为 torch.Size([32, 1, 100])

torch.Size([32, 1, 100])
